In [1]:
from azure.cosmos import exceptions, CosmosClient, PartitionKey

endpoint = 'https://piqdata.documents.azure.com:443/'
key = 'LzqfuGbrcnIDkeDbsErfXjtm4gmG2RMP5jAys5ifWcgwmOdM7mCXqsYoOusybZHHspsOVNhU0QhACNCqUvoBtg=='

In [2]:
client = CosmosClient(endpoint, key)
database_name = 'RetailDemo'
database = client.create_database_if_not_exists(database_name)
print(f'Database {database_name} was created.')

container_name = 'WebsiteData'
container = database.create_container_if_not_exists(
    id=container_name,
    partition_key=PartitionKey(path="/CartID"),
    offer_throughput=400
)
print(f'Container {container_name} was created.')

Database RetailDemo was created.
Container WebsiteData was created.


In [3]:
# {
#     "CartID":5399,
#     "Action":"Viewed",
#     "Item":"Cosmos T-shirt",
#     "Price":350,
#     "UserName":"Demo.User10",
#     "Country":"Iceland",
#     "EventDate":"2015-06-25T00:00:00",
#     "Year":2015,"Latitude":-66.8673,
#     "Longitude":-29.8214,
#     "Address":"852 Modesto Loop, Port Ola, Iceland",
#     "id":"00ffd39c-7e98-4451-9b91-b2bcf2f9a32d"
#  }

# Import data from Azure blob
import urllib.request, json

flag = False

if flag:
    with urllib.request.urlopen("https://cosmosnotebooksdata.blob.core.windows.net/notebookdata/websiteData.json") as url:
        data = json.loads(url.read().decode())
        
    print("Importing data...")

    for event in data:
        try:
            container.upsert_item(body=event)
        except errors.CosmosHttpRequestError as e:
            raise

query = "SELECT VALUE COUNT(1) FROM c"
result = list(container.query_items(query, enable_cross_partition_query=True))

print(f"Container with id {container.id} contains {result[0]} items.")


Container with id WebsiteData contains 2654 items.


In [9]:
import pandas as pd

# Read a sample
query = "SELECT c.Action, c.Price as ItemRevenue, c.Country, c.Item FROM c"

items = container.query_items(query=query, enable_cross_partition_query=True)

df_cosmos = pd.DataFrame(items)

df_cosmos.head(10)

,Action,ItemRevenue,Country,Item
0,Viewed,9.00,Tunisia,Black Tee
1,Viewed,19.99,Antigua and Barbuda,Flannel Shirt
2,Added,3.75,Guinea-Bissau,Socks
3,Viewed,3.75,Guinea-Bissau,Socks
4,Viewed,55.00,Czech Republic,Rainjacket
5,Viewed,350.00,Iceland,Cosmos T-shirt
6,Added,19.99,Syrian Arab Republic,Button-Up Shirt
7,Viewed,19.99,Syrian Arab Republic,Button-Up Shirt
8,Viewed,33.00,Tuvalu,Red Top
9,Viewed,14.00,Cape Verde,Flip Flop Shoes


In [16]:
df_revenue = df_cosmos.groupby("Country").sum().reset_index().sort_values("ItemRevenue", ascending=False)
display(df_revenue.head(5))

,Country,ItemRevenue
202,Svalbard & Jan Mayen Islands,2362.45
216,Tunisia,2297.96
7,Antigua and Barbuda,1834.93
130,Marshall Islands,1752.48
129,Malta,1691.43


In [17]:
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.palettes import Spectral3
from bokeh.transform import factor_cmap
from bokeh.models import ColumnDataSource, FactorRange

# Get the top 10 items as an array
top_10_items = df_cosmos[df_cosmos['Action']=='Purchased'].groupby('Item').size().sort_values(ascending=False)[:10].index.values.tolist()

# Filter our data to only these 10 items
df_top10 = df_cosmos[df_cosmos['Item'].isin(top_10_items)]

# Group by Item and Action, sorting by event count
df_top10_sorted = df_top10.groupby(['Item', 'Action']).count().rename(columns={'Country':'ResultCount'}, inplace=False).reset_index().sort_values(['Item', 'ResultCount'], ascending = False).set_index(['Item', 'Action'])

# Get sorted X-axis values - this way, we can display the funnel of view -> add -> purchase
x_axis_values = df_top10_sorted.index.values.tolist()

group = df_top10_sorted.groupby(['Item', 'Action'])

# Specifiy colors for X axis
index_cmap = factor_cmap('Item_Action', palette=Spectral3, factors=sorted(df_top10.Action.unique()), start=1, end=2)

# Create the plot

p = figure(plot_width=1200, plot_height=500, title="Conversion rate of items from View -> Add to cart -> Purchase", x_range=FactorRange(*x_axis_values), toolbar_location=None, tooltips=[("Number of events", "@ResultCount_max"), ("Item, Action", "@Item_Action")])

p.vbar(x='Item_Action', top='ItemRevenue_max', width=1, source=group, line_color="white", fill_color=index_cmap, )

#Configure how the plot looks
p.y_range.start = 0
p.x_range.range_padding = 0.05
p.xgrid.grid_line_color = None
p.xaxis.major_label_orientation = 1.2
p.outline_line_color = "black"
p.xaxis.axis_label = "Item"
p.yaxis.axis_label = "Count"

#Display figure inline in Jupyter Notebook.
output_notebook()

#Display figure.
show(p)

Loading BokehJS ...